In [36]:
# Standard libraries
import time
import re

# External libraries
import lightkurve as lk
import pandas as pd
import numpy as np
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
from astroquery.mast import Catalogs

In [2]:
# Initial parameters
target_data_file = "../Data/RawTargetData/Interferometric Targets - TimWhite.csv"

In [3]:
reference_sheet = pd.read_csv(target_data_file)
reference_sheet

,Name,HD,RA,Dec,Vmag,Sp. Type,Interferometry Ref.,KIC,Notes,dhuber's list,Boyajian+2013,Neither
0,PAVO Limb Darkening Sample,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,eta Cas,4614,00 49 06.2907161,+57 48 54.67574,3.440,F9V+M0-V,White et al. in prep.,NaN,NaN,Y,Y,0.0
2,ups And,9826,01 36 47.84216,+41 24 19.6443,4.100,F9V,White et al. in prep.,NaN,NaN,Y,Y,0.0
3,tau Cet,10700,01 44 04.0834226,-15 56 14.926552,3.500,G8V,more data needed,NaN,NaN,Y,Y,0.0
4,eps Eri,22049,03 32 55.8449634,-09 27 29.731165,3.730,K2V,more data needed,NaN,NaN,Y,N,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
132,eta Sco,155203,17 12 09.1956531,-43 14 21.090526,3.330,F5IV,Rains et al. 2020,NaN,NaN,N,N,1.0
133,bet Aql,188512,19 55 18.7925630,+06 24 24.342501,3.710,G8IV,Rains et al. 2020,NaN,NaN,N,Y,0.0
134,HR 7221,177389,19 09 52.8681650178,-68 25 28.021373885,5.305,K0IV,Rains et al. 2020,NaN,NaN,N,N,1.0
135,gam Pav,203608,21 26 26.6048372,-65 21 58.314484,4.220,F9VFe-1.4CH-0.7,unpublished?,NaN,NaN,N,N,1.0


In [4]:
reduced_targets = reference_sheet[reference_sheet["dhuber's list"] != "Y"][reference_sheet["Boyajian+2013"] != "Y"]
to_drop = []
for idx in reduced_targets.index.values:
    if isinstance(reduced_targets.loc[idx, "HD"], float) or not isinstance(reduced_targets.loc[idx, "KIC"], float):
        to_drop.append(idx)
reduced_targets.drop(to_drop, inplace=True)
reduced_targets.reset_index(inplace=True, drop=True)
reduced_targets

,Name,HD,RA,Dec,Vmag,Sp. Type,Interferometry Ref.,KIC,Notes,dhuber's list,Boyajian+2013,Neither
0,mu Leo,85503,09 52 45.8165435,+26 00 25.031882,3.880,K2IIIbCN1Ca1,White et al. in prep.,NaN,NaN,N,N,1.0
1,46 LMi,94264,10 53 18.7048735,+34 12 53.537492,3.830,K0+III-IV,White et al. in prep.,NaN,NaN,N,N,1.0
2,bet Boo,133208,15 01 56.7623786,+40 23 26.040607,3.520,G8IIIaFe-0.5,more data needed,NaN,NaN,N,N,1.0
3,del Boo,135722,15 15 30.1629518,+33 18 53.392558,3.490,G8IIIFe-1,White et al. in prep.,NaN,NaN,N,N,1.0
4,eps Oph,146791,16 18 19.2897431,-04 41 33.034466,3.230,G9.5IIIbFe-0.5,more data needed,NaN,NaN,N,N,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
72,HD 131977,131977,14 57 28.0008928545,-21 24 55.726897561,5.720,K4V,Rains et al. 2020,NaN,NaN,N,N,1.0
73,eta Sco,155203,17 12 09.1956531,-43 14 21.090526,3.330,F5IV,Rains et al. 2020,NaN,NaN,N,N,1.0
74,HR 7221,177389,19 09 52.8681650178,-68 25 28.021373885,5.305,K0IV,Rains et al. 2020,NaN,NaN,N,N,1.0
75,gam Pav,203608,21 26 26.6048372,-65 21 58.314484,4.220,F9VFe-1.4CH-0.7,unpublished?,NaN,NaN,N,N,1.0


In [5]:
target_list = pd.DataFrame({
    "Star": [],
    "Main ID": [],
    "TIC": [],
    "Sectors": [],
    "Pmix": [],
    "ATL numax (muHz)": [],
    "TIC 0.8M numax (muHz)": [],
    "TIC 1.2M numax (muHz)": [],
    "TIC 2.4M numax (muHz)": [],
    "RA": [],
    "DEC": [],
    "SpT": [],
    "V": [],
    "H": [],
    "K": [],
    "Diam": [],
    "Err": [],
    "Reference": [],
    "Instrument": [],
    "Fast": []
})
target_list

,Star,Main ID,TIC,Sectors,Pmix,ATL numax (muHz),TIC 0.8M numax (muHz),TIC 1.2M numax (muHz),TIC 2.4M numax (muHz),RA,DEC,SpT,V,H,K,Diam,Err,Reference,Instrument,Fast


In [6]:
headings = {}
headings["PAVO Limb Darkening Sample"] = "PAVO"
headings["PAVO - Kepler Sample"] = "PAVO"
headings["PAVO - Gaia Benchmarks Sample"] = "PAVO"
headings["PAVO - 'Retired A Stars' Sample"] = "PAVO"
headings["VLTI - Adam's Sample"] = "VLTI"
headings

{'PAVO Limb Darkening Sample': 'PAVO',
 'PAVO - Kepler Sample': 'PAVO',
 'PAVO - Gaia Benchmarks Sample': 'PAVO',
 "PAVO - 'Retired A Stars' Sample": 'PAVO',
 "VLTI - Adam's Sample": 'VLTI'}

In [7]:
references = {}
instruments = {}
for idx in reference_sheet.index.values:
    if isinstance(reference_sheet.loc[idx, "HD"], float):
        current_instrument = headings[reference_sheet.loc[idx, "Name"]]
        continue
    if "GJ" in reference_sheet.loc[idx, "HD"]:
        star_hd = reference_sheet.loc[idx, "HD"].replace("GJ", "GJ ")
    else:
        star_hd = "HD " + reference_sheet.loc[idx, "HD"]
    references[star_hd] = reference_sheet.loc[idx, "Interferometry Ref."]
    instruments[star_hd] = current_instrument

In [8]:
"""Add HD and Main IDs"""
for idx in reference_sheet.index.values:
    if np.isnan(reference_sheet.loc[idx, "Vmag"]):
        continue
    if "GJ" in str(reference_sheet.loc[idx, "HD"]):
        target_list.loc[idx, "Star"] = str(reference_sheet.loc[idx, "HD"]).replace("GJ", "GJ ")
    else:
        target_list.loc[idx, "Star"] = "HD " + str(reference_sheet.loc[idx, "HD"])
    target_list.loc[idx, "Main ID"] = reference_sheet.loc[idx, "Name"]
target_list

,Star,Main ID,TIC,Sectors,Pmix,ATL numax (muHz),TIC 0.8M numax (muHz),TIC 1.2M numax (muHz),TIC 2.4M numax (muHz),RA,DEC,SpT,V,H,K,Diam,Err,Reference,Instrument,Fast
1,HD 4614,eta Cas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HD 9826,ups And,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HD 10700,tau Cet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HD 22049,eps Eri,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,HD 23249,del Eri,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,HD 155203,eta Sco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,HD 188512,bet Aql,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,HD 177389,HR 7221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,HD 203608,gam Pav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
"""Filter targets with TESS data"""
# Query using lightkurve to get TIC and sectors
tic = []
sectors = []
to_add = []

start = time.time()
for idx in target_list.index.values:
    print(f"Star {idx} out of {len(target_list)}")
    star = target_list.loc[idx, "Star"]
    search = lk.search_lightcurvefile(star, mission="TESS")
    if len(search) == 0:
        continue
    else:
        to_add.append(idx)
    
    # Go through search results
    current_sectors = []
    current_target_name = ""
    # Make sure the target is the same for all sectors
    for (sector, target_name) in search.table[["observation", "target_name"]]:
        if current_target_name == "":
            current_target_name = target_name
        if target_name != current_target_name:
            continue
        current_sectors.append(sector.replace("TESS Sector ", ""))
    
    # Add new sectors and TIC
    target_list.loc[idx, "TIC"] = str(int(current_target_name))
    target_list.loc[idx, "Sectors"] = ", ".join(current_sectors)

target_list = target_list.loc[to_add]
target_list.reset_index(drop=True, inplace=True)

elapsed = time.time() - start
print(f"This took {elapsed//60} minutes and {elapsed%60} seconds.")

Star 0 out of 83
Star 1 out of 83
Star 2 out of 83
Star 3 out of 83
Star 4 out of 83
Star 5 out of 83
Star 6 out of 83
Star 7 out of 83
Star 8 out of 83
Star 9 out of 83
Star 10 out of 83
Star 11 out of 83
Star 12 out of 83
Star 13 out of 83
Star 14 out of 83
Star 15 out of 83
Star 16 out of 83
Star 17 out of 83
Star 18 out of 83
Star 19 out of 83
Star 20 out of 83
Star 21 out of 83
Star 22 out of 83
Star 23 out of 83
Star 24 out of 83
Star 25 out of 83
Star 26 out of 83
Star 27 out of 83
Star 28 out of 83
Star 29 out of 83
Star 30 out of 83
Star 31 out of 83
Star 32 out of 83
Star 33 out of 83
Star 34 out of 83
Star 35 out of 83
Star 36 out of 83
Star 37 out of 83
Star 38 out of 83
Star 39 out of 83
Star 40 out of 83
Star 41 out of 83
Star 42 out of 83
Star 43 out of 83
Star 44 out of 83
Star 45 out of 83
Star 46 out of 83
Star 47 out of 83
Star 48 out of 83
Star 49 out of 83
Star 50 out of 83
Star 51 out of 83
Star 52 out of 83
Star 53 out of 83
Star 54 out of 83
Star 55 out of 83
St

In [10]:
custom_simbad = Simbad()
custom_simbad.add_votable_fields(
    "sptype",
    "flux(V)", "flux_bibcode(V)",
    "flux(H)", "flux_bibcode(H)",
    "flux(K)", "flux_bibcode(K)"
)

In [11]:
query_delay = 3.0
minimum_time = query_delay*len(target_list["Star"])
print(f"Querying will at least {minimum_time//60} minutes and {minimum_time%60} seconds.")

# SIMBAD querying
for idx in target_list.index.values:
    # Query until success
    query = None
    while query is None:
        try:
            query = custom_simbad.query_object(target_list.loc[idx, "Star"])
            print(f"Successfully queried #{idx+1}: {target_list.loc[idx, 'Star']}")
        except:
            print(f"Error: Retrying connection...", end="\r")
            time.sleep(5.0)
    
    # Process query
    main_id = re.sub("\s\s+", " ", query["MAIN_ID"][0].decode("utf-8"))
    ra = query["RA"][0]
    dec = query["DEC"][0]
    spt = query["SP_TYPE"][0].decode("utf-8")
    # Fluxes
    v_mag = query["FLUX_V"][0]
    h_mag = query["FLUX_H"][0]
    k_mag = query["FLUX_K"][0]

    # Set new values
    target_list.loc[idx, "Main ID"] = re.sub("^\*\s*", "", main_id)
    target_list.loc[idx, "RA"] = ra
    target_list.loc[idx, "DEC"] = dec
    target_list.loc[idx, "SpT"] = spt
    target_list.loc[idx, "V"] = v_mag
    target_list.loc[idx, "H"] = h_mag
    target_list.loc[idx, "K"] = k_mag

    time.sleep(query_delay)
target_list

Querying will at least 3.0 minutes and 51.0 seconds.
Successfully queried #1: HD 4614
Successfully queried #2: HD 9826
Successfully queried #3: HD 10700
Successfully queried #4: HD 22049
Successfully queried #5: HD 30652
Successfully queried #6: HD 48737
Successfully queried #7: HD 85503
Successfully queried #8: HD 121370
Successfully queried #9: HD 133208
Successfully queried #10: HD 137759
Successfully queried #11: HD 148856
Successfully queried #12: HD 150997
Successfully queried #13: HD 150680
Successfully queried #14: HD 161797
Successfully queried #15: HD 163588
Successfully queried #16: HD 163993
Successfully queried #17: HD 169414
Successfully queried #18: HD 181276
Successfully queried #19: HD 198149
Successfully queried #20: HD 219619
Successfully queried #21: HD 222404
Successfully queried #22: HD 173701
Successfully queried #23: HD 174880
Successfully queried #24: HD 175740
Successfully queried #25: HD 175884
Successfully queried #26: HD 177153
Successfully queried #27: HD 

,Star,Main ID,TIC,Sectors,Pmix,ATL numax (muHz),TIC 0.8M numax (muHz),TIC 1.2M numax (muHz),TIC 2.4M numax (muHz),RA,DEC,SpT,V,H,K,Diam,Err,Reference,Instrument,Fast
0,HD 4614,eta Cas,445258206.0,"17, 18, 24",NaN,NaN,NaN,NaN,NaN,00 49 06.2907,+57 48 54.675,F9V+M0-V,3.440,2.020,1.990,NaN,NaN,NaN,NaN,NaN
1,HD 9826,ups And,189576919.0,17,NaN,NaN,NaN,NaN,NaN,01 36 47.8421,+41 24 19.644,F9V,4.100,2.957,2.860,NaN,NaN,NaN,NaN,NaN
2,HD 10700,tau Cet,419015728.0,"3, 30, 30",NaN,NaN,NaN,NaN,NaN,01 44 04.0834,-15 56 14.926,G8V,3.500,1.720,1.680,NaN,NaN,NaN,NaN,NaN
3,HD 22049,eps Eri,118572803.0,"4, 31, 31",NaN,NaN,NaN,NaN,NaN,03 32 55.8449,-09 27 29.731,K2V,3.730,1.750,1.670,NaN,NaN,NaN,NaN,NaN
4,HD 30652,pi.03 Ori,399665349.0,"5, 32",NaN,NaN,NaN,NaN,NaN,04 49 50.4109,+06 57 40.588,F6V,3.190,2.150,2.050,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,HD 209100,eps Ind,231698181.0,1,NaN,NaN,NaN,NaN,NaN,22 03 21.6542,-56 47 09.537,K5V,4.690,2.349,2.237,NaN,NaN,NaN,NaN,NaN
73,HD 155203,eta Sco,382600457.0,12,NaN,NaN,NaN,NaN,NaN,17 12 09.1956,-43 14 21.090,F5IV,3.330,2.380,2.340,NaN,NaN,NaN,NaN,NaN
74,HD 177389,HD 177389,382276084.0,"13, 27",NaN,NaN,NaN,NaN,NaN,19 09 52.8681,-68 25 28.021,K0IV,5.305,3.117,2.994,NaN,NaN,NaN,NaN,NaN
75,HD 203608,gam Pav,265488188.0,"1, 13, 27, 27",NaN,NaN,NaN,NaN,NaN,21 26 26.6048,-65 21 58.314,F9VFe-1.4CH-0.7,4.220,2.980,2.910,NaN,NaN,NaN,NaN,NaN


In [19]:
def sort_ids(data_table: pd.DataFrame) -> pd.DataFrame:
    """Sort IDs by numerical order with GJ IDs first then HD IDs last.

    Parameters
    ----------
    data_table : pd.DataFrame
        the table to be sorted. Requires a Star column.
    
    Returns
    -------
    data_table : pd.DataFrame
        sorted data table
    """

    def sorting_key(entry):
        """Sorting key"""
        return entry[1]
    
    star_names = list(enumerate(data_table["Star"]))
    gj_id = []
    hd_id = []
    for star in star_names:
        if "GJ" in star[1]:
            gj_id.append(star)
        elif "HD" in star[1]:
            hd_id.append(star)

    # Sort GJ ids by number
    gj_indices = sorted([(idx, int(re.search("\d+", x[1]).group(0))) for idx, x in enumerate(gj_id)], key=sorting_key)
    gj_id = [gj_id[idx] for idx, _ in gj_indices]
    hd_indices = sorted([(idx, int(re.search("\d+", x[1]).group(0))) for idx, x in enumerate(hd_id)], key=sorting_key)
    hd_id = [hd_id[idx] for idx, _ in hd_indices]

    sorted_names = gj_id + hd_id
    sorted_indices = [idx for idx, _ in sorted_names]

    # Finally sorted
    data_table = data_table.loc[sorted_indices].reset_index(drop=True)
    return data_table


In [21]:
target_list = sort_ids(target_list)
target_list

,Star,Main ID,TIC,Sectors,Pmix,ATL numax (muHz),TIC 0.8M numax (muHz),TIC 1.2M numax (muHz),TIC 2.4M numax (muHz),RA,DEC,SpT,V,H,K,Diam,Err,Reference,Instrument,Fast
0,GJ 551,NAME Proxima Centauri,388857263.0,"11, 12",NaN,NaN,NaN,NaN,NaN,14 29 42.9451,-62 40 46.170,M5.5Ve,11.130,4.835,4.384,NaN,NaN,NaN,NaN,NaN
1,HD 4614,eta Cas,445258206.0,"17, 18, 24",NaN,NaN,NaN,NaN,NaN,00 49 06.2907,+57 48 54.675,F9V+M0-V,3.440,2.020,1.990,NaN,NaN,NaN,NaN,NaN
2,HD 6833,HD 6833,331212009.0,"17, 18",NaN,NaN,NaN,NaN,NaN,01 09 52.2649,+54 44 20.273,K0IIIbFe-2,6.740,3.980,3.860,NaN,NaN,NaN,NaN,NaN
3,HD 7927,phi Cas,332680754.0,18,NaN,NaN,NaN,NaN,NaN,01 20 04.9146,+58 13 53.812,F0Ia,4.980,3.120,3.090,NaN,NaN,NaN,NaN,NaN
4,HD 9826,ups And,189576919.0,17,NaN,NaN,NaN,NaN,NaN,01 36 47.8421,+41 24 19.644,F9V,4.100,2.957,2.860,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,HD 207978,15 Peg,326202925.0,15,NaN,NaN,NaN,NaN,NaN,21 52 29.9167,+28 47 36.751,F2V,5.530,4.441,4.332,NaN,NaN,NaN,NaN,NaN
73,HD 209100,eps Ind,231698181.0,1,NaN,NaN,NaN,NaN,NaN,22 03 21.6542,-56 47 09.537,K5V,4.690,2.349,2.237,NaN,NaN,NaN,NaN,NaN
74,HD 216174,HD 216174,444179899.0,"16, 17, 24",NaN,NaN,NaN,NaN,NaN,22 49 46.3147,+55 54 09.990,K1III,5.439,2.922,2.625,NaN,NaN,NaN,NaN,NaN
75,HD 219619,HD 219619,175398506.0,"28, 29",NaN,NaN,NaN,NaN,NaN,23 17 27.3070,-39 52 08.639,F0V,8.610,7.654,7.627,NaN,NaN,NaN,NaN,NaN


In [22]:
# Fix Proxima Centauri name
for idx in target_list.index.values:
    target_list.loc[idx, "Main ID"] = target_list.loc[idx, "Main ID"].replace("NAME Proxima Centauri", "Proxima Centauri")

In [29]:
"""Add References and Instruments"""
references = {}
instruments = {}
for idx in reference_sheet.index.values:
    if isinstance(reference_sheet.loc[idx, "HD"], float):
        current_instrument = headings[reference_sheet.loc[idx, "Name"]]
        continue
    if "GJ" in reference_sheet.loc[idx, "HD"]:
        star_hd = reference_sheet.loc[idx, "HD"].replace("GJ", "GJ ")
    else:
        star_hd = "HD " + reference_sheet.loc[idx, "HD"]
    references[star_hd] = reference_sheet.loc[idx, "Interferometry Ref."]
    instruments[star_hd] = current_instrument

for idx in target_list.index.values:
    target_list.loc[idx, "Reference"] = references[target_list.loc[idx, "Star"]]
    target_list.loc[idx, "Instrument"] = instruments[target_list.loc[idx, "Star"]]
target_list

,Star,Main ID,TIC,Sectors,Pmix,ATL numax (muHz),TIC 0.8M numax (muHz),TIC 1.2M numax (muHz),TIC 2.4M numax (muHz),RA,DEC,SpT,V,H,K,Diam,Err,Reference,Instrument,Fast
0,GJ 551,Proxima Centauri,388857263.0,"11, 12",NaN,NaN,NaN,NaN,NaN,14 29 42.9451,-62 40 46.170,M5.5Ve,11.130,4.835,4.384,NaN,NaN,unpublished?,VLTI,NaN
1,HD 4614,eta Cas,445258206.0,"17, 18, 24",NaN,NaN,NaN,NaN,NaN,00 49 06.2907,+57 48 54.675,F9V+M0-V,3.440,2.020,1.990,NaN,NaN,White et al. in prep.,PAVO,NaN
2,HD 6833,HD 6833,331212009.0,"17, 18",NaN,NaN,NaN,NaN,NaN,01 09 52.2649,+54 44 20.273,K0IIIbFe-2,6.740,3.980,3.860,NaN,NaN,Karovicova et al. 2020,PAVO,NaN
3,HD 7927,phi Cas,332680754.0,18,NaN,NaN,NaN,NaN,NaN,01 20 04.9146,+58 13 53.812,F0Ia,4.980,3.120,3.090,NaN,NaN,unpublished,PAVO,NaN
4,HD 9826,ups And,189576919.0,17,NaN,NaN,NaN,NaN,NaN,01 36 47.8421,+41 24 19.644,F9V,4.100,2.957,2.860,NaN,NaN,White et al. in prep.,PAVO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,HD 207978,15 Peg,326202925.0,15,NaN,NaN,NaN,NaN,NaN,21 52 29.9167,+28 47 36.751,F2V,5.530,4.441,4.332,NaN,NaN,unpublished,PAVO,NaN
73,HD 209100,eps Ind,231698181.0,1,NaN,NaN,NaN,NaN,NaN,22 03 21.6542,-56 47 09.537,K5V,4.690,2.349,2.237,NaN,NaN,Rains et al. 2020,VLTI,NaN
74,HD 216174,HD 216174,444179899.0,"16, 17, 24",NaN,NaN,NaN,NaN,NaN,22 49 46.3147,+55 54 09.990,K1III,5.439,2.922,2.625,NaN,NaN,no data,PAVO,NaN
75,HD 219619,HD 219619,175398506.0,"28, 29",NaN,NaN,NaN,NaN,NaN,23 17 27.3070,-39 52 08.639,F0V,8.610,7.654,7.627,NaN,NaN,no data yet,PAVO,NaN


In [33]:
"""Add fast cadences"""
target_list_append = target_list[0:0]
for idx in target_list.index.values:
    search = lk.search_lightcurvefile(target_list.loc[idx, "Star"], mission="TESS")
    
    fast_sectors = []

    sectors = []
    current_target = None
    for sector, target, filename in search.table[["observation", "target_name", "productFilename"]]:            
        if current_target is None:
            current_target = target
        elif current_target != target:
            continue

        if "fast" in filename:
            fast_sectors.append(sector.replace("TESS Sector ", ""))
        else:
            sectors.append(sector.replace("TESS Sector ", ""))
    
    target_list.loc[idx, "Sectors"] = ", ".join(sectors)

    if len(fast_sectors) > 0:
        current_fast_index = len(target_list_append)
        target_list_append.loc[current_fast_index] = target_list.loc[idx]
        target_list_append.loc[current_fast_index, "Sectors"] = ", ".join(fast_sectors)
        target_list_append.loc[current_fast_index, "Fast"] = True
    print(f"Star {idx} out of {len(target_list)}")
target_list = pd.concat([target_list, target_list_append], ignore_index=True)
target_list = sort_ids(target_list)
target_list.reset_index(drop=True, inplace=True)
target_list

Star 0 out of 77
Star 1 out of 77
Star 2 out of 77
Star 3 out of 77
Star 4 out of 77
Star 5 out of 77
Star 6 out of 77
Star 7 out of 77
Star 8 out of 77
Star 9 out of 77
Star 10 out of 77
Star 11 out of 77
Star 12 out of 77
Star 13 out of 77
Star 14 out of 77
Star 15 out of 77
Star 16 out of 77
Star 17 out of 77
Star 18 out of 77
Star 19 out of 77
Star 20 out of 77
Star 21 out of 77
Star 22 out of 77
Star 23 out of 77
Star 24 out of 77
Star 25 out of 77
Star 26 out of 77
Star 27 out of 77
Star 28 out of 77
Star 29 out of 77
Star 30 out of 77
Star 31 out of 77
Star 32 out of 77
Star 33 out of 77
Star 34 out of 77
Star 35 out of 77
Star 36 out of 77
Star 37 out of 77
Star 38 out of 77
Star 39 out of 77
Star 40 out of 77
Star 41 out of 77
Star 42 out of 77
Star 43 out of 77
Star 44 out of 77
Star 45 out of 77
Star 46 out of 77
Star 47 out of 77
Star 48 out of 77
Star 49 out of 77
Star 50 out of 77
Star 51 out of 77
Star 52 out of 77
Star 53 out of 77
Star 54 out of 77
Star 55 out of 77
St

,Star,Main ID,TIC,Sectors,Pmix,ATL numax (muHz),TIC 0.8M numax (muHz),TIC 1.2M numax (muHz),TIC 2.4M numax (muHz),RA,DEC,SpT,V,H,K,Diam,Err,Reference,Instrument,Fast
0,GJ 551,Proxima Centauri,388857263.0,"11, 12",NaN,NaN,NaN,NaN,NaN,14 29 42.9451,-62 40 46.170,M5.5Ve,11.130,4.835,4.384,NaN,NaN,unpublished?,VLTI,NaN
1,HD 4614,eta Cas,445258206.0,"17, 18, 24",1.000,2953.201,NaN,NaN,NaN,00 49 06.2907,+57 48 54.675,F9V+M0-V,3.440,2.020,1.990,NaN,NaN,White et al. in prep.,PAVO,NaN
2,HD 6833,HD 6833,331212009.0,"17, 18",NaN,NaN,NaN,NaN,NaN,01 09 52.2649,+54 44 20.273,K0IIIbFe-2,6.740,3.980,3.860,NaN,NaN,Karovicova et al. 2020,PAVO,NaN
3,HD 7927,phi Cas,332680754.0,18,NaN,NaN,NaN,NaN,NaN,01 20 04.9146,+58 13 53.812,F0Ia,4.980,3.120,3.090,NaN,NaN,unpublished,PAVO,NaN
4,HD 9826,ups And,189576919.0,17,1.000,1336.345,NaN,NaN,NaN,01 36 47.8421,+41 24 19.644,F9V,4.100,2.957,2.860,NaN,NaN,White et al. in prep.,PAVO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,HD 207978,15 Peg,326202925.0,15,0.796,1749.984,NaN,NaN,NaN,21 52 29.9167,+28 47 36.751,F2V,5.530,4.441,4.332,NaN,NaN,unpublished,PAVO,NaN
79,HD 209100,eps Ind,231698181.0,1,NaN,NaN,NaN,NaN,NaN,22 03 21.6542,-56 47 09.537,K5V,4.690,2.349,2.237,NaN,NaN,Rains et al. 2020,VLTI,NaN
80,HD 216174,HD 216174,444179899.0,"16, 17, 24",NaN,NaN,NaN,NaN,NaN,22 49 46.3147,+55 54 09.990,K1III,5.439,2.922,2.625,NaN,NaN,no data,PAVO,NaN
81,HD 219619,HD 219619,175398506.0,"28, 29",0.405,892.229,NaN,NaN,NaN,23 17 27.3070,-39 52 08.639,F0V,8.610,7.654,7.627,NaN,NaN,no data yet,PAVO,NaN


In [31]:
# Search ATL catalogue
v = Vizier(columns=["TIC", "Pmix", "numax"], catalog="J/ApJS/241/12")

for idx in target_list.index.values:
    star = target_list.loc[idx, "Star"]
    search = v.query_object(star)
    if len(search) == 0:
        pmix = np.nan
        numax = np.nan
    else:
        search = search[0]
        pmix = float(search["Pmix"][0])
        numax = float(search["numax"][0])
    target_list.loc[idx, "Pmix"] = pmix
    target_list.loc[idx, "ATL numax (muHz)"] = numax
    print(f"Star {idx} out of {len(target_list)}")

target_list

Star 0 out of 77
Star 1 out of 77
Star 2 out of 77
Star 3 out of 77
Star 4 out of 77
Star 5 out of 77
Star 6 out of 77
Star 7 out of 77
Star 8 out of 77
Star 9 out of 77
Star 10 out of 77
Star 11 out of 77
Star 12 out of 77
Star 13 out of 77
Star 14 out of 77
Star 15 out of 77
Star 16 out of 77
Star 17 out of 77
Star 18 out of 77
Star 19 out of 77
Star 20 out of 77
Star 21 out of 77
Star 22 out of 77
Star 23 out of 77
Star 24 out of 77
Star 25 out of 77
Star 26 out of 77
Star 27 out of 77
Star 28 out of 77
Star 29 out of 77
Star 30 out of 77
Star 31 out of 77
Star 32 out of 77
Star 33 out of 77
Star 34 out of 77
Star 35 out of 77
Star 36 out of 77
Star 37 out of 77
Star 38 out of 77
Star 39 out of 77
Star 40 out of 77
Star 41 out of 77
Star 42 out of 77
Star 43 out of 77
Star 44 out of 77
Star 45 out of 77
Star 46 out of 77
Star 47 out of 77
Star 48 out of 77
Star 49 out of 77
Star 50 out of 77
Star 51 out of 77
Star 52 out of 77
Star 53 out of 77
Star 54 out of 77
Star 55 out of 77
St

,Star,Main ID,TIC,Sectors,Pmix,ATL numax (muHz),TIC 0.8M numax (muHz),TIC 1.2M numax (muHz),TIC 2.4M numax (muHz),RA,DEC,SpT,V,H,K,Diam,Err,Reference,Instrument,Fast
0,GJ 551,Proxima Centauri,388857263.0,"11, 12",NaN,NaN,NaN,NaN,NaN,14 29 42.9451,-62 40 46.170,M5.5Ve,11.130,4.835,4.384,NaN,NaN,unpublished?,VLTI,NaN
1,HD 4614,eta Cas,445258206.0,"17, 18, 24",1.000,2953.201,NaN,NaN,NaN,00 49 06.2907,+57 48 54.675,F9V+M0-V,3.440,2.020,1.990,NaN,NaN,White et al. in prep.,PAVO,NaN
2,HD 6833,HD 6833,331212009.0,"17, 18",NaN,NaN,NaN,NaN,NaN,01 09 52.2649,+54 44 20.273,K0IIIbFe-2,6.740,3.980,3.860,NaN,NaN,Karovicova et al. 2020,PAVO,NaN
3,HD 7927,phi Cas,332680754.0,18,NaN,NaN,NaN,NaN,NaN,01 20 04.9146,+58 13 53.812,F0Ia,4.980,3.120,3.090,NaN,NaN,unpublished,PAVO,NaN
4,HD 9826,ups And,189576919.0,17,1.000,1336.345,NaN,NaN,NaN,01 36 47.8421,+41 24 19.644,F9V,4.100,2.957,2.860,NaN,NaN,White et al. in prep.,PAVO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,HD 207978,15 Peg,326202925.0,15,0.796,1749.984,NaN,NaN,NaN,21 52 29.9167,+28 47 36.751,F2V,5.530,4.441,4.332,NaN,NaN,unpublished,PAVO,NaN
73,HD 209100,eps Ind,231698181.0,1,NaN,NaN,NaN,NaN,NaN,22 03 21.6542,-56 47 09.537,K5V,4.690,2.349,2.237,NaN,NaN,Rains et al. 2020,VLTI,NaN
74,HD 216174,HD 216174,444179899.0,"16, 17, 24",NaN,NaN,NaN,NaN,NaN,22 49 46.3147,+55 54 09.990,K1III,5.439,2.922,2.625,NaN,NaN,no data,PAVO,NaN
75,HD 219619,HD 219619,175398506.0,"28, 29",0.405,892.229,NaN,NaN,NaN,23 17 27.3070,-39 52 08.639,F0V,8.610,7.654,7.627,NaN,NaN,no data yet,PAVO,NaN


In [40]:
"""TIC estimated numax"""
def estimate_numax(tic_number, mass):
    result = Catalogs.query_criteria(catalog="Tic", ID=tic_number)
    t_eff = result["Teff"][0]
    radius = result["rad"][0]
    # Stefan-Boltzmann law
    luminosity = radius**2 * (t_eff/5777.0)**4
    # Estimate numax from scaling relation
    numax = 3100.0 * mass * (t_eff/5777.0)**3.5 / luminosity
    return numax

for idx in target_list.index.values:
    spt = target_list.loc[idx, "SpT"]
    if "F" in spt or "G" in spt or "K" in spt:
        tic_number = target_list.loc[idx, "TIC"]
        target_list.loc[idx, "TIC 0.8M numax (muHz)"] = estimate_numax(tic_number, 0.8)
        target_list.loc[idx, "TIC 1.2M numax (muHz)"] = estimate_numax(tic_number, 1.2)
        target_list.loc[idx, "TIC 2.4M numax (muHz)"] = estimate_numax(tic_number, 2.4)
    print(f"Star {idx} out of {len(target_list)}")
target_list

Star 0 out of 83
Star 1 out of 83
Star 2 out of 83
Star 3 out of 83
Star 4 out of 83
Star 5 out of 83
Star 6 out of 83
Star 7 out of 83
Star 8 out of 83
Star 9 out of 83
Star 10 out of 83
Star 11 out of 83
Star 12 out of 83
Star 13 out of 83
Star 14 out of 83
Star 15 out of 83
Star 16 out of 83
Star 17 out of 83
Star 18 out of 83
Star 19 out of 83
Star 20 out of 83
Star 21 out of 83
Star 22 out of 83
Star 23 out of 83
Star 24 out of 83
Star 25 out of 83
Star 26 out of 83
Star 27 out of 83
Star 28 out of 83
Star 29 out of 83
Star 30 out of 83
Star 31 out of 83
Star 32 out of 83
Star 33 out of 83
Star 34 out of 83
Star 35 out of 83
Star 36 out of 83
Star 37 out of 83
Star 38 out of 83
Star 39 out of 83
Star 40 out of 83
Star 41 out of 83
Star 42 out of 83
Star 43 out of 83
Star 44 out of 83
Star 45 out of 83
Star 46 out of 83
Star 47 out of 83
Star 48 out of 83
Star 49 out of 83
Star 50 out of 83
Star 51 out of 83
Star 52 out of 83
Star 53 out of 83
Star 54 out of 83
Star 55 out of 83
St

,Star,Main ID,TIC,Sectors,Pmix,ATL numax (muHz),TIC 0.8M numax (muHz),TIC 1.2M numax (muHz),TIC 2.4M numax (muHz),RA,DEC,SpT,V,H,K,Diam,Err,Reference,Instrument,Fast
0,GJ 551,Proxima Centauri,388857263,"11, 12",NaN,NaN,NaN,NaN,NaN,14 29 42.9451,-62 40 46.170,M5.5Ve,11.130,4.835,4.384,NaN,NaN,unpublished?,VLTI,NaN
1,HD 4614,eta Cas,445258206,"17, 18, 24",1.000,2953.201,1996.186389,2994.279584,5988.559168,00 49 06.2907,+57 48 54.675,F9V+M0-V,3.440,2.020,1.990,NaN,NaN,White et al. in prep.,PAVO,NaN
2,HD 6833,HD 6833,331212009,"17, 18",NaN,NaN,8.045205,12.067807,24.135614,01 09 52.2649,+54 44 20.273,K0IIIbFe-2,6.740,3.980,3.860,NaN,NaN,Karovicova et al. 2020,PAVO,NaN
3,HD 7927,phi Cas,332680754,18,NaN,NaN,0.024251,0.036376,0.072752,01 20 04.9146,+58 13 53.812,F0Ia,4.980,3.120,3.090,NaN,NaN,unpublished,PAVO,NaN
4,HD 9826,ups And,189576919,17,1.000,1336.345,900.772625,1351.158938,2702.317876,01 36 47.8421,+41 24 19.644,F9V,4.100,2.957,2.860,NaN,NaN,White et al. in prep.,PAVO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,HD 207978,15 Peg,326202925,15,0.796,1749.984,930.430955,1395.646432,2791.292864,21 52 29.9167,+28 47 36.751,F2V,5.530,4.441,4.332,NaN,NaN,unpublished,PAVO,NaN
79,HD 209100,eps Ind,231698181,1,NaN,NaN,5096.680393,7645.020590,15290.041180,22 03 21.6542,-56 47 09.537,K5V,4.690,2.349,2.237,NaN,NaN,Rains et al. 2020,VLTI,NaN
80,HD 216174,HD 216174,444179899,"16, 17, 24",NaN,NaN,6.510844,9.766266,19.532532,22 49 46.3147,+55 54 09.990,K1III,5.439,2.922,2.625,NaN,NaN,no data,PAVO,NaN
81,HD 219619,HD 219619,175398506,"28, 29",0.405,892.229,411.013056,616.519584,1233.039168,23 17 27.3070,-39 52 08.639,F0V,8.610,7.654,7.627,NaN,NaN,no data yet,PAVO,NaN


In [45]:
# Set False to nan values in Fast column
for idx in target_list.index.values:
    if isinstance(target_list.loc[idx, "Fast"], float):
        target_list.loc[idx, "Fast"] = False
target_list

,Star,Main ID,TIC,Sectors,Pmix,ATL numax (muHz),TIC 0.8M numax (muHz),TIC 1.2M numax (muHz),TIC 2.4M numax (muHz),RA,DEC,SpT,V,H,K,Diam,Err,Reference,Instrument,Fast
0,GJ 551,Proxima Centauri,388857263,"11, 12",NaN,NaN,NaN,NaN,NaN,14 29 42.9451,-62 40 46.170,M5.5Ve,11.130,4.835,4.384,NaN,NaN,unpublished?,VLTI,False
1,HD 4614,eta Cas,445258206,"17, 18, 24",1.000,2953.201,1996.186389,2994.279584,5988.559168,00 49 06.2907,+57 48 54.675,F9V+M0-V,3.440,2.020,1.990,NaN,NaN,White et al. in prep.,PAVO,False
2,HD 6833,HD 6833,331212009,"17, 18",NaN,NaN,8.045205,12.067807,24.135614,01 09 52.2649,+54 44 20.273,K0IIIbFe-2,6.740,3.980,3.860,NaN,NaN,Karovicova et al. 2020,PAVO,False
3,HD 7927,phi Cas,332680754,18,NaN,NaN,0.024251,0.036376,0.072752,01 20 04.9146,+58 13 53.812,F0Ia,4.980,3.120,3.090,NaN,NaN,unpublished,PAVO,False
4,HD 9826,ups And,189576919,17,1.000,1336.345,900.772625,1351.158938,2702.317876,01 36 47.8421,+41 24 19.644,F9V,4.100,2.957,2.860,NaN,NaN,White et al. in prep.,PAVO,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,HD 207978,15 Peg,326202925,15,0.796,1749.984,930.430955,1395.646432,2791.292864,21 52 29.9167,+28 47 36.751,F2V,5.530,4.441,4.332,NaN,NaN,unpublished,PAVO,False
79,HD 209100,eps Ind,231698181,1,NaN,NaN,5096.680393,7645.020590,15290.041180,22 03 21.6542,-56 47 09.537,K5V,4.690,2.349,2.237,NaN,NaN,Rains et al. 2020,VLTI,False
80,HD 216174,HD 216174,444179899,"16, 17, 24",NaN,NaN,6.510844,9.766266,19.532532,22 49 46.3147,+55 54 09.990,K1III,5.439,2.922,2.625,NaN,NaN,no data,PAVO,False
81,HD 219619,HD 219619,175398506,"28, 29",0.405,892.229,411.013056,616.519584,1233.039168,23 17 27.3070,-39 52 08.639,F0V,8.610,7.654,7.627,NaN,NaN,no data yet,PAVO,False


In [46]:
target_list.to_csv("../Data/PlotData/twt_plot_data.csv", index=False)